<a href="https://colab.research.google.com/github/lhfp98/git-bootcamp/blob/main/Gibson_report2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install tabula-py
!pip install tabula-py

# Import the read_pdf function from tabula
from tabula import read_pdf

# URL to the PDF file containing the tabular data
URL = "/gibson_2020/2020 GIBSON GAS - WEEK (16).pdf"

# Use tabula to read the PDF file from the given URL
tabular_data = read_pdf(URL, pages='all', lattice=True)  # For well-defined grids
tabular_data = read_pdf(URL, pages='all', multiple_tables=True)



In [5]:
tabular_data[5]

,Mts,Unnamed: 0,Unnamed: 1,Unnamed: 2,US $/pmt,vs Previous week
0,"44,000",LPG,AG,Japan,$61.071,Up $4.357
1,"6,500",BTD/PPL,Brazil,USG-Caribs,145-155,Under pressure
2,"6,500",C2/BTD,Brazil,CJK,300-310,Under pressure
3,"6,500",Ethylene,USG,N.W.E,175-185,Under pressure
4,"6,500",Ethylene,N.W.E.,CJK,290-300,Under pressure
5,"6,500",Ethylene,MEG,SE Asia,100-110,Under pressure
6,"6,500",Ethylene,MEG,CJK,145-155,Under pressure
7,"6,500",Butadiene,N.W.E,CJK,280-290,Under pressure
8,"6,500",Propylene,USG,N.W.E,145-155,Under pressure
9,"1,600",Propylene,Yosu,E.China,54,Unchanged


In [ ]:
import tabula
import pandas as pd
import os

def extract_tables(pdf_path):
    # Attempt to extract tables using tabula
    try:
        # Using stream mode based on your table layout; adjust pages as needed
        tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, stream=True)
        return tables
    except Exception as e:
        print(f"Failed to extract tables from {pdf_path}: {str(e)}")
        return None

def validate_tables(tables, expected_columns):
    # Check if any of the tables match the expected structure
    valid_tables = []
    for table in tables:
        if all(col in table.columns for col in expected_columns):
            valid_tables.append(table)
    return valid_tables

# Specify your PDF directory and expected columns based on your sample images
pdf_directory = '/gibson'
expected_columns = ['Mts', 'US $/pmt', 'vs Previous week']

# Process each PDF in the directory
results = {}
for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        full_path = os.path.join(pdf_directory, pdf_file)
        tables = extract_tables(full_path)
        if tables:
            valid_tables = validate_tables(tables, expected_columns)
            if valid_tables:
                results[pdf_file] = valid_tables
            else:
                results[pdf_file] = "No valid tables found"
        else:
            results[pdf_file] = "No tables extracted"

# Output the results
for key, value in results.items():
    print(f"File: {key}, Result: {value}")


Nov 20, 2024 9:07:47 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 20, 2024 9:07:47 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 20, 2024 9:07:47 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 20, 2024 9:07:48 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Nov 20, 2024 9:08:27 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



File: 2019 GIBSON GAS - WEEK (51).pdf, Result: [       Mts Unnamed: 0 Unnamed: 1   Unnamed: 2 US $/pmt vs Previous week
0   44,000        LPG         AG        Japan  $68.143      Down $0.500
1    6,500    BTD/PPL     Brazil   USG-Caribs  140-150        Unchanged
2    6,500     C2/BTD     Brazil          CJK  290-300        Unchanged
3    6,500   Ethylene        USG        N.W.E  165-175        Unchanged
4    6,500   Ethylene     N.W.E.          CJK  280-290        Unchanged
5    6,500   Ethylene        MEG      SE Asia  100-110        Unchanged
6    6,500   Ethylene        MEG          CJK  145-155        Unchanged
7    6,500  Butadiene      N.W.E          CJK  270-280        Unchanged
8    6,500  Propylene        USG        N.W.E  140-150        Unchanged
9    1,600  Propylene       Yosu      E.China       54        Unchanged
10   1,500  Propylene      S.E.A      E.China      162        Unchanged
11   4,000     Butane      EC UK      Morocco       64          Down $1
12   1,800     B

## Working code 2019

In [25]:
!pip install tabula-py pytesseract pdf2image

import tabula
import pandas as pd
import os
from datetime import datetime
import re
from dateutil import parser
import pytesseract
from pdf2image import convert_from_path

def clean_date_string(raw_string):
    """
    Cleans a raw string to extract a date, handling cases where it includes prefixes like 'Week XX | '.
    """
    try:
        # Extract date-like parts, ignoring any preceding "Week XX | " text
        match = re.search(r'(\d{1,2}\s+[a-zA-Z]+\s+\d{4})', raw_string)
        if match:
            return match.group(1)  # Extract matched date portion
        else:
            return None  # No valid date found
    except Exception as e:
        print(f"Error cleaning date string '{raw_string}': {e}")
        return None

def extract_date_from_pdf(pdf_path):
    """
    Attempts to extract a date from the specified area in the PDF using Tabula. Falls back to OCR if Tabula fails.
    """
    area = [0, 400, 60, 600]  # Define the area for Tabula
    try:
        # Attempt to extract data using Tabula
        tables = tabula.read_pdf(pdf_path, area=area, pages=1, multiple_tables=True, pandas_options={'header': None})
        if tables and not tables[0].empty:
            raw_date_string = str(tables[0].iloc[0, 0])  # Assume date is in the first cell

            if pd.isna(raw_date_string) or raw_date_string in ["", None]:
                print(f"Empty or invalid date string found in {pdf_path}")
                return None

            print(f"Raw date string from Tabula: {raw_date_string}")  # Debugging output

            cleaned_date_string = clean_date_string(raw_date_string)
            print(f"Cleaned date string from Tabula: {cleaned_date_string}")  # Debugging output

            if cleaned_date_string:
                try:
                    date_object = parser.parse(cleaned_date_string)
                    return date_object.strftime('%Y-%m-%d')
                except parser.ParserError as pe:
                    print(f"Parser error on cleaned string '{cleaned_date_string}': {pe}")
                    return None
            else:
                return None
        else:
            print(f"No valid data found in specified area of {pdf_path}")
    except Exception as e:
        print(f"Error extracting date using Tabula from {pdf_path}: {e}")

    # Fallback to OCR if Tabula fails
    print(f"Falling back to OCR for {pdf_path}")
    try:
        pages = convert_from_path(pdf_path)
        text = pytesseract.image_to_string(pages[0])  # Extract text from the first page
        print(f"OCR extracted text: {text}")  # Debugging output

        # Extract date using regex
        match = re.search(r'\d{1,2}\s+[a-zA-Z]+\s+\d{4}', text)
        if match:
            date_string = match.group(0)
            print(f"OCR date string: {date_string}")
            try:
                date_object = parser.parse(date_string)
                return date_object.strftime('%Y-%m-%d')
            except parser.ParserError as pe:
                print(f"Parser error on OCR date string '{date_string}': {pe}")
                return None
        else:
            print(f"No valid date found in OCR text for {pdf_path}")
            return None
    except Exception as e:
        print(f"Error extracting date using OCR from {pdf_path}: {e}")
        return None

def extract_and_validate_tables(pdf_path, expected_columns):
    """
    Extracts and validates tables with specified columns from a PDF.
    """
    try:
        tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, stream=True)
        valid_tables = []
        for table in tables:
            if all(col in table.columns for col in expected_columns):
                valid_tables.append(table)
        return valid_tables
    except Exception as e:
        print(f"Error extracting tables from {pdf_path}: {e}")
        return []

def process_pdfs(pdf_directory, expected_columns, output_csv_path):
    """
    Processes all PDFs in the specified directory, extracting tables and their associated dates.
    """
    all_tables = []
    for pdf_file in os.listdir(pdf_directory):
        if pdf_file.endswith('.pdf'):
            full_path = os.path.join(pdf_directory, pdf_file)
            print(f"Processing {pdf_file}...")
            date_extracted = extract_date_from_pdf(full_path)
            if date_extracted:
                tables = extract_and_validate_tables(full_path, expected_columns)
                for table in tables:
                    table['Report Date'] = date_extracted
                    all_tables.append(table)
            else:
                print(f"Failed to extract date from {pdf_file}")

    # Concatenate and save results
    if all_tables:
        combined_df = pd.concat(all_tables, ignore_index=True)
        combined_df.to_csv(output_csv_path, index=False)
        print(f"Data successfully written to {output_csv_path}")
        return combined_df
    else:
        print("No data extracted.")
        return None

# Example usage
pdf_directory = '/gibson_2020'  # Update to your directory
expected_columns = ['Mts', 'US $/pmt', 'vs Previous week']  # Columns to validate
output_csv_path = 'combined_data_2020.csv'  # Output file path
combined_data = process_pdfs(pdf_directory, expected_columns, output_csv_path)

if combined_data is not None:
    print(combined_data)
else:
    print("No data to display.")


Processing 2020 GIBSON GAS - WEEK (30).pdf...
No valid data found in specified area of /gibson_2020/2020 GIBSON GAS - WEEK (30).pdf
Falling back to OCR for /gibson_2020/2020 GIBSON GAS - WEEK (30).pdf
Error extracting date using OCR from /gibson_2020/2020 GIBSON GAS - WEEK (30).pdf: Unable to get page count. Is poppler installed and in PATH?
Failed to extract date from 2020 GIBSON GAS - WEEK (30).pdf
Processing 2020 GIBSON GAS - WEEK (16).pdf...
No valid data found in specified area of /gibson_2020/2020 GIBSON GAS - WEEK (16).pdf
Falling back to OCR for /gibson_2020/2020 GIBSON GAS - WEEK (16).pdf
Error extracting date using OCR from /gibson_2020/2020 GIBSON GAS - WEEK (16).pdf: Unable to get page count. Is poppler installed and in PATH?
Failed to extract date from 2020 GIBSON GAS - WEEK (16).pdf
No data extracted.
No data to display.


ERROR:tabula.backend:Error from tabula-java:
Error: /gibson (Is a directory)


Error extracting date from /gibson: Command '['java', '-Dfile.encoding=UTF8', '-jar', '/usr/local/lib/python3.10/dist-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar', '--pages', '1', '--area', '10,350,100,600', '--format', 'JSON', '/gibson']' returned non-zero exit status 1.
Extracted Date: None
WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:00:00 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
WARNING: New fonts found, font cache will be re-built
Nov 20, 2024 1:00:00 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
WARNING: Building on-disk font cache, this may take a while
Nov 20, 2024 1:00:00 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
WARNING: Finished building on-disk font cache, found 17 fonts

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:01:22 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:01:24 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:01:26 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:02:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:02:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:02:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:03:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:03:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:03:07 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

Error extracting date from /gibson/2020 GIBSON GAS - WEEK (36).pdf: expected string or bytes-like object
Failed to extract date from 2020 GIBSON GAS - WEEK (36).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:04:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:04:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:04:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:04:44 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:04:47 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:07:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:07:43 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:07:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 1:07:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 1:07:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 1:07:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Verdana'
Nov 20, 2024 1:07:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'CenturyGothic-Bold'
Nov 20, 2024 1:07:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:07:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 1:07:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 1:07:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 1:07:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Verdana'
Nov 20, 2024 1:07:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'CenturyGothic-Bold'
Nov 20, 2024 1:07:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:07:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 1:07:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 1:07:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 1:07:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Verdana'
Nov 20, 2024 1:07:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'CenturyGothic-Bold'
Nov 20, 2024 1:07:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:07:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 1:07:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 1:07:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 1:07:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 1:07:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 1:07:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 1:07:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Verdana'
Nov 20, 2024 1:07:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:07:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 1:07:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 1:07:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 1:07:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'CenturyGothic'
Nov 20, 2024 1:07:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Italic' for 'SegoeUI-Italic'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'CenturyGothic'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Italic' for 'SegoeUI-Italic'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'CenturyGothic-Bold'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri-Light'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 1:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri-Light'

Error extracting date from /gibson/2021 GIBSON GAS - WEEK (12).pdf: cannot use a string pattern on a bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (12).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:09:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:09:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:09:44 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

Error extracting date from /gibson/2020 GIBSON GAS - WEEK (45).pdf: expected string or bytes-like object
Failed to extract date from 2020 GIBSON GAS - WEEK (45).pdf
Error extracting date from /gibson/2021 GIBSON GAS - WEEK (11).pdf: expected string or bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (11).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:17:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:17:08 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:17:10 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:17:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:17:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:17:53 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:17:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:19:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:19:32 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:20:32 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:20:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:20:36 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:23:25 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:23:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:25:31 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:25:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:25:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:26:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:26:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:27:18 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:27:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

Error extracting date from /gibson/2020 GIBSON GAS - WEEK (30).pdf: cannot use a string pattern on a bytes-like object
Failed to extract date from 2020 GIBSON GAS - WEEK (30).pdf
Error extracting date from /gibson/2021 GIBSON GAS - WEEK (15).pdf: cannot use a string pattern on a bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (15).pdf
Error extracting date from /gibson/2020 GIBSON GAS - WEEK (44).pdf: expected string or bytes-like object
Failed to extract date from 2020 GIBSON GAS - WEEK (44).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:32:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:32:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:32:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:35:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:35:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:35:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:35:52 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:35:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:36:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:36:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

Error extracting date from /gibson/2020 GIBSON GAS - WEEK (16).pdf: expected string or bytes-like object
Failed to extract date from 2020 GIBSON GAS - WEEK (16).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:38:27 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:38:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:42:37 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:42:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:42:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:43:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:43:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:46:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:46:07 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:53:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:53:51 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:55:26 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:55:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:55:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

Error extracting date from /gibson/2021 GIBSON GAS - WEEK (9).pdf: expected string or bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (9).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:56:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 1:56:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:57:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:57:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 1:57:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

Error extracting date from /gibson/2021 GIBSON GAS - WEEK (16).pdf: cannot use a string pattern on a bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (16).pdf
Error extracting date from /gibson/2021 GIBSON GAS - WEEK (5).pdf: expected string or bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (5).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:11:24 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:11:26 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:11:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:11:44 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:11:47 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:14:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:15:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

Error extracting date from /gibson/2021 GIBSON GAS - WEEK (6).pdf: cannot use a string pattern on a bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (6).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:15:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 2:15:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 2:15:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 2:15:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Verdana'
Nov 20, 2024 2:15:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'CenturyGothic-Bold'
Nov 20, 2024 2:15:55 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:15:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 2:15:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 2:15:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 2:15:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Verdana'
Nov 20, 2024 2:15:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'CenturyGothic-Bold'
Nov 20, 2024 2:15:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:16:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 2:16:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 2:16:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 2:16:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Verdana'
Nov 20, 2024 2:16:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'CenturyGothic-Bold'
Nov 20, 2024 2:16:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:16:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 2:16:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 2:16:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 2:16:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 2:16:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 2:16:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 2:16:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Verdana'
Nov 20, 2024 2:16:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:16:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 2:16:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 2:16:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 2:16:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'CenturyGothic'
Nov 20, 2024 2:16:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Italic' for 'SegoeUI-Italic'
Nov 20, 2024 2:16:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 2:16:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 2:16:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 2:16:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'CenturyGothic'
Nov 20, 2024 2:16:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Italic' for 'SegoeUI-Italic'
Nov 20, 2024 2:16:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'CenturyGothic-Bold'
Nov 20, 2024 2:16:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri-Light'
Nov 20, 2024 2:16:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri'
Nov 20, 2024 2:16:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'SegoeUI'
Nov 20, 2024 2:16:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans-Bold' for 'SegoeUI-Bold'
Nov 20, 2024 2:16:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'Calibri-Light'

Error extracting date from /gibson/2020 GIBSON GAS - WEEK (24).pdf: expected string or bytes-like object
Failed to extract date from 2020 GIBSON GAS - WEEK (24).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:17:47 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:17:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:18:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:18:26 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:18:27 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

Error extracting date from /gibson/2021 GIBSON GAS - WEEK (17).pdf: expected string or bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (17).pdf
Error extracting date from /gibson/2021 GIBSON GAS - WEEK (20).pdf: expected string or bytes-like object
Failed to extract date from 2021 GIBSON GAS - WEEK (20).pdf
WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:22:07 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:22:09 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:24:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:24:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'

WARNING:tabula.backend:Got stderr: Nov 20, 2024 2:25:34 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'
Nov 20, 2024 2:25:36 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
WARNING: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'


In [ ]:
combined_data


,Mts,Unnamed: 0,Unnamed: 1,Unnamed: 2,US $/pmt,vs Previous week,Report Date
0,"44,000",LPG,AG,Japan,77.143,Up +8.500,2022-09-22
1,"44,000",LPG,USG,Japan,125.000,Up +6.571,2022-09-22
2,"11,500",Ethylene,Houston,CJK,370-390,Unchanged,2022-09-22
3,"6,500",Ethylene,USG,N.W.E,240-260,Unchanged,2022-09-22
4,"6,500",Ethylene,N.W.E.,CJK,360-370,Unchanged,2022-09-22
...,...,...,...,...,...,...,...
4712,"4,000",Butane,EC UK,Morocco,66,Unchanged,Date format not recognized
4713,"1,800",Butane,Tees,ARA,44,Up $2,Date format not recognized
4714,"1,800",Butane,Tees,Lisbon,91,Up $2,Date format not recognized
4715,"1,600",Propane,Tees,Brest,70,Up $2,Date format not recognized


In [ ]:
combined_data['Report Date'] = pd.to_datetime(combined_data['Report Date'], errors='coerce')
# 'errors='coerce'' will turn any non-convertible values into NaT (Not a Time), which helps avoid crashes due to format issues.

unique_dates = combined_data['Report Date'].dropna().unique()
unique_dates_df = pd.DataFrame(unique_dates, columns=['Unique Dates'])
sorted_dates = unique_dates_df.sort_values(by='Unique Dates')
print(sorted_dates)


    Unique Dates
51    2017-01-12
158   2017-01-19
70    2017-01-26
7     2017-02-09
79    2017-02-16
..           ...
128   2023-08-31
137   2023-09-07
187   2023-09-14
159   2023-09-21
65    2023-09-28

[245 rows x 1 columns]
